# Modificación del dataset de películas

Importación de módulos a utilizar:

In [3]:
import os
import csv
import itertools
import string
from collections import Counter
import random

Funciones definidas:

In [4]:
def elijo_palabras(secuencia):
    """Retorna una lista de hasta 3 items elegidos al azar de una lista"""
    if len(secuencia)<=2:
        return secuencia
    else:
        return random.sample(secuencia,3)

Se abre archivo con el que se va a trabajar.

Se deben obtener las 100 palabras que más aparecen en los overviews.

Además, hay que descartar las películas que no tienen “overview” y las películas cuyo idioma original tenga más de 2 caracteres.

In [20]:
ruta=os.getcwd()
arch = 'mymoviedb.csv'
arch_nuevo = 'data_set_peliculas.csv'
try:
    with open(os.path.join(ruta,'..','csv',arch),'r',encoding='utf-8') as archivo: ## los dos puntos sirven para ir al directorio anterior
        
        reader = csv.reader(archivo,delimiter=',')
        
        encabezado_peliculas = next(reader) #encabezado del archivo original
               
        descarte = filter(lambda x: x[2]!='' and len(x[6])<3,reader)
        
        filas_peliculas = list(descarte) #guardo información del archivo original en una lista de listas
        
except FileNotFoundError:
    print(f"No se encuentra el archivo {arch}")      
        

Se agrupan los overview en una sola lista:

In [7]:
overview = [fila[2] for fila in filas_peliculas]

Se separan las palabras de la columna del overview:

In [8]:
overview_map = map(lambda x: x.split(' '),overview)

Se guarda en una variable (que se utilizará más adelante) la lista de listas de palabras por overview:

In [9]:
palabras_por_overview = [x for x in overview_map]

Se agrupan todas las palabras en una sóla lista:

In [10]:
palabras_overview = list(itertools.chain.from_iterable(palabras_por_overview))

Las palabras en minúscula y sin signos de puntuación:

In [11]:
palabras = map(lambda x: x.lower().strip(string.punctuation),palabras_overview)

Se toman las 100 palabras más comunes de todos los overviews combinados:

In [12]:
#Se obtiene lista de tuplas, donde el primer elemento es la palabra y el segundo las veces que aparece.
top_100 = Counter(palabras).most_common(100)

#Se obtiene una lista con los elementos pertenecientes a la primera posición de cada tupla.
top_100 = [x[0] for x in top_100]

print(top_100)     

['the', 'a', 'to', 'and', 'of', 'in', 'his', 'is', 'with', 'her', 'an', 'he', 'for', 'on', 'that', 'their', 'as', 'when', 'by', 'who', 'from', 'but', 'they', 'she', 'are', 'has', 'after', 'into', 'at', 'him', 'life', 'new', 'one', 'young', '', 'it', 'up', 'out', 'world', 'be', 'must', 'two', 'this', 'will', 'all', 'have', 'family', 'them', 'find', 'man', 'only', 'love', 'while', 'where', 'friends', 'time', 'years', 'finds', 'story', 'about', 'not', 'woman', 'help', 'school', 'get', 'home', 'back', 'was', 'can', 'own', 'mysterious', 'what', 'now', 'against', 'take', 'group', 'which', 'save', 'way', 'before', 'more', 'girl', 'first', 'other', 'himself', 'soon', 'been', 'father', 'through', 'war', 'over', 'become', 'lives', 'most', 'becomes', 'town', 'being', 'down', 'takes', 'together']


En el nuevo archivo la columna del overview no debe poseer las palabras pertenecientes al top_100.

Además debe tener hasta 3 palabras.

Se eliminan del overview las palabras pertenecientes al top 100 y las palabras que son simplemente el signo "-" ya que no sirven para el juego:

In [16]:
palabras_sin_top_100 = list(map(lambda y: [x.lower().strip(string.punctuation) for x in y if (x.lower().strip(string.punctuation) not in top_100 and x!='—' and x!='' )],palabras_por_overview))

Se eligen hasta 3 palabras por overview:

In [17]:
overview_reducido = map(lambda x: elijo_palabras(x),palabras_sin_top_100)
overview_final = list(map(lambda x: ';'.join(x),overview_reducido))

Creo el nuevo archivo:

In [19]:
try:
    with open(os.path.join(ruta,'archivos',arch_nuevo),'x',encoding='utf-8') as nuevo:
        
        #Se ecribe encabezado en el nuevo archivo
        x = encabezado_peliculas
        lista_encabezado = [x[7],x[6],x[0],x[5],x[2],x[1]]
        nuevo.write(','.join(lista_encabezado))
        
        linea = filas_peliculas
        
        for i in range(len(linea)):
            columnas = ",".join(['"' + linea[i][7] + '"',linea[i][6],linea[i][0],linea[i][5],overview_final[i],linea[i][1]])
            nuevo.write('\n' + columnas)
        
except FileExistsError:
    print(f"El archivo {arch_nuevo} ya existe")